In [3]:
import json
import numpy as np
import random
import nltk
# import svgling
import os
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    "maxent_ne_chunker",
    "words",
])

[nltk_data] Downloading package names to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to


True

In [2]:
# qa_dataset = testset0+testset1+testset2+testset3+testset4
qa_dataset = json.load(open('../../data/CausalQA/causalqa.json', 'r', encoding='utf-8'))
qa_dataset[0]

{'qid': 432,
 'context': "The global restaurant sector has come under pressure as several markets have restricted service in an effort to curb the spread of the coronavirus. Restrictions vary by geography, but even in markets where carry-out and drive-thru orders are still permitted, we expect uneven guest counts into 2021. Nevertheless, we believe investors should prioritize those players that possess the scale to be more aggressive on pricing near term (value-oriented players tend to outperform during economic shocks), give their customers greater access through robust digital ordering, delivery, and drive-thru capabilities, and have healthy balance sheets (both at the corporate and franchisee level) when looking for restaurant industry investment opportunities. While we believe COVID-19 disruptions may result in some restructuring situations and restaurant closures in 2020 among RBI's franchisee base, the operational strategies behind its core brands--Burger King, Tim Hortons, and P

### Question Type Template

In [3]:
print('Number of qa pairs: ', len(qa_dataset))
question_list = []
for qa in qa_dataset: 
    question_list.append(qa['question'])

Number of qa pairs:  24462


In [5]:
what = {'count':0, 'list':[]}
why = {'count':0, 'list':[]}
what_willhappen = {'count':0, 'list':[]}
why_useful = {'count':0, 'list':[]}
other = {'count':0, 'list':[]}

for query in question_list:
    query = query.capitalize()
    if 'What will happen' in query:
        what_willhappen['count'] += 1
        what_willhappen['list'].append(query)
    elif 'What' in query:
        what['count'] += 1
        what['list'].append(query)
    elif ('Why' in query) and ('useful' in query):
        why_useful['count'] += 1
        why_useful['list'].append(query)
    elif 'Why' in query:
        why['count'] += 1
        why['list'].append(query)
    else:
        other['count'] += 1
        other['list'].append(query)
print('Why: ',why['count'], 'What: ',what['count'], 'What will happen',what_willhappen['count'], 'Why useful',why_useful['count'], 'Other', other['count'])
print(why['count']+what['count']+what_willhappen['count']+why_useful['count']+other['count'])
print(len(question_list))

Why:  7705 What:  5241 What will happen 7694 Why useful 3822 Other 0
24462
24462


### Add Qtype template to dataset

In [47]:
def count_words(dataset):
    words = {}
    for i in range(len(dataset)):
        for word in dataset[i].split():
            if word not in words:
                words[word]=1
            else:
                words[word]+=1
    return words

def sort_wordCounter(words_dict):
    # input: a dict of words, key:word, value:times of word shown in the context
    # output: a list sorted by the value of dict
    a = dict(sorted(words_dict.items(), key=lambda x:x[1], reverse=True))
    return a


In [48]:
what = {'count':0, 'list':[]}
why = {'count':0, 'list':[]}
what_willhappen = {'count':0, 'list':[]}
why_useful = {'count':0, 'list':[]}
other = {'count':0, 'list':[]}

for qa in qa_dataset:
    query = qa['question']
    query = query.capitalize()
    (start, end) = qa['answer']
    answer_text = qa['context'][start:end]
    context_list = qa['context'].split('. ')
    for sentence in context_list:
        if answer_text in sentence:
            break
    
    if 'What will happen' in query:
        what_willhappen['count'] += 1
        what_willhappen['list'].append(sentence)
    elif 'What' in query:
        what['count'] += 1
        what['list'].append(sentence)
    elif ('Why' in query) and ('useful' in query):
        why_useful['count'] += 1
        why_useful['list'].append(sentence)
    elif 'Why' in query:
        why['count'] += 1
        why['list'].append(sentence)
    else:
        other['count'] += 1
        other['list'].append(sentence)

In [49]:
why_wordcount = sort_wordCounter(count_words(why['list']))
what_wordcount = sort_wordCounter(count_words(what['list']))
whatwillhappen_wordcount = sort_wordCounter(count_words(what_willhappen['list']))
whyuseful_wordcount = sort_wordCounter(count_words(why_useful['list']))
other_wordcount = sort_wordCounter(count_words(other['list']))

wordcount_list = [why_wordcount, what_wordcount, whatwillhappen_wordcount,whyuseful_wordcount, other_wordcount]
print('Num of type of questions: ', len(wordcount_list))

Num of type of questions:  5


In [50]:
def pmi(wordCount,qtype, wordcount_list=wordcount_list):
    pmi_dict = {}
    for word in wordCount:
        # word occurance more than quenstion number, the word must be a very common word,
        # like 'the', 'a', 'it', etc. These words does not help answering qestions

        if wordCount[word]>qtype['count']:
            continue
        # word occurance less than 5% quenstion number
        # it should be a random occurance, we consider there is no relation between such word and the question
        if wordCount[word]<qtype['count']/20:
            break
        else:
            num_word = 0
            vocab_num = 0
            for one_wordcount in wordcount_list:
                if word in one_wordcount:
                    num_word += one_wordcount[word]
                vocab_num += len(one_wordcount)
            p_word = num_word / vocab_num
            conditional_p_word = wordCount[word]/len(wordCount)
            
            pmi_dict[word] = np.log(conditional_p_word/p_word)
    
    return pmi_dict

def get_top50_prompt_words(why_wordcount,qtype):    
    #input: word count dict
    #output: top 50 prompt word list of corresponds to the word count
    why_word_pmiDict = pmi(why_wordcount, qtype) 
    sort_why_pmi = sort_wordCounter(why_word_pmiDict)
    top50_whyPromptWords_list = list(sort_why_pmi)[0:10]
    return top50_whyPromptWords_list

In [51]:
top50_whyPromptWords_list = get_top50_prompt_words(why_wordcount, why)
top50_whatPromptWords_list = get_top50_prompt_words(what_wordcount, what)
top50_whatwillhappenPromptWords_list = get_top50_prompt_words(whatwillhappen_wordcount, what_willhappen)
top50_whyusefulPromptWords_list = get_top50_prompt_words(whyuseful_wordcount, why_useful)
top50_otherPromptWords_list = get_top50_prompt_words(other_wordcount, other)


In [52]:
top50_whyusefulPromptWords_list

['through',
 'should',
 'firm',
 'into',
 'its',
 'believe',
 'it',
 'cost',
 'can',
 'customers']

In [53]:
def generate_Qtype_template(qa_dataset):
    # Random given prompt word for Qtype

    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break
    
    for qa in qa_dataset:
        query = qa['question']
        query = query.capitalize()
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break

        # question type
            
        if "What will happen" in query:
            template_01 = 'To answer a "What will happen" question, you need to look for "{}"'.format(top50_whatwillhappenPromptWords_list[random.randint(0,len(top50_whatwillhappenPromptWords_list)-1)])   
        elif 'What' in query:
            template_01 = 'To answer a "What" question, you need to look for "{}"'.format(top50_whatPromptWords_list[random.randint(0,len(top50_whatPromptWords_list)-1)])   
        elif ('Why' in query) and ('useful' in query):
            template_01 = 'To answer a "Why useful" question, you need to look for "{}"'.format(top50_whyusefulPromptWords_list[random.randint(0,len(top50_whyusefulPromptWords_list)-1)])   
        elif 'Why' in query:
            template_01 = 'To answer a "Why" question, you need to look for "{}"'.format(top50_whyPromptWords_list[random.randint(0,len(top50_whyPromptWords_list)-1)])   


        template = template_01
    
        qa['template'] = template
    
    return qa_dataset

In [54]:
f = json.load(open('../../data/CausalQA/causalqa.json'))
withPrompt = generate_Qtype_template(f)
json.dump(withPrompt, open('../../data/CausalQA/causalQA_Qtype.json', 'w', encoding='utf-8'))

In [15]:
sample = json.load(open('../../data/CausalQA/causalQA_Qtype.json', 'r', encoding='utf-8'))
sample[0]

{'qid': 432,
 'context': "The global restaurant sector has come under pressure as several markets have restricted service in an effort to curb the spread of the coronavirus. Restrictions vary by geography, but even in markets where carry-out and drive-thru orders are still permitted, we expect uneven guest counts into 2021. Nevertheless, we believe investors should prioritize those players that possess the scale to be more aggressive on pricing near term (value-oriented players tend to outperform during economic shocks), give their customers greater access through robust digital ordering, delivery, and drive-thru capabilities, and have healthy balance sheets (both at the corporate and franchisee level) when looking for restaurant industry investment opportunities. While we believe COVID-19 disruptions may result in some restructuring situations and restaurant closures in 2020 among RBI's franchisee base, the operational strategies behind its core brands--Burger King, Tim Hortons, and P

In [16]:
dev_len = int(len(sample)/10)
train = sample[:-2*dev_len]
dev = sample[-2*dev_len:-1*dev_len]
test = sample[-1*dev_len:]
json.dump(train, open('../../data/CausalQA/Qtype_causal/causal_train.json', 'w', encoding='utf-8'))
json.dump(dev, open('../../data/CausalQA/Qtype_causal/causal_dev.json', 'w', encoding='utf-8'))
json.dump(test, open('../../data/CausalQA/Qtype_causal/causal_test.json', 'w', encoding='utf-8'))

In [22]:
sample = json.load(open('../../data/CausalQA/original/causal_train.json', 'r', encoding='utf-8'))
sample[0]

{'qid': 432,
 'context': "The global restaurant sector has come under pressure as several markets have restricted service in an effort to curb the spread of the coronavirus. Restrictions vary by geography, but even in markets where carry-out and drive-thru orders are still permitted, we expect uneven guest counts into 2021. Nevertheless, we believe investors should prioritize those players that possess the scale to be more aggressive on pricing near term (value-oriented players tend to outperform during economic shocks), give their customers greater access through robust digital ordering, delivery, and drive-thru capabilities, and have healthy balance sheets (both at the corporate and franchisee level) when looking for restaurant industry investment opportunities. While we believe COVID-19 disruptions may result in some restructuring situations and restaurant closures in 2020 among RBI's franchisee base, the operational strategies behind its core brands--Burger King, Tim Hortons, and P

In [20]:
sample = json.load(open('../../data/CausalQA/causalQA.json', 'r', encoding='utf-8'))
dev_len = int(len(sample)/10)
train = sample[:-2*dev_len]
dev = sample[-2*dev_len:-1*dev_len]
test = sample[-1*dev_len:]
json.dump(train, open('../../data/CausalQA/original/causal_train.json', 'w', encoding='utf-8'))
json.dump(dev, open('../../data/CausalQA/original/causal_dev.json', 'w', encoding='utf-8'))
json.dump(test, open('../../data/CausalQA/original/causal_test.json', 'w', encoding='utf-8'))

### Add phrase template to dataset

In [90]:
def generate_phrase_template_for_domain(qa_dataset):
    pattern = """
        NP: {++}
        NPs: {++}
        Ns: {++}
        """
    pattenParse = nltk.RegexpParser(pattern)

    # count=0
    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')

        tokens = nltk.word_tokenize(query)
        tagged = nltk.pos_tag(tokens)
        cs = pattenParse.parse(tagged)
        phrase_template = '"'
        word_template = '"'
        candidates = []
        for i in cs:
            if type(i) != tuple: 
                for word in i:
                    phrase_template += word[0]+' ' 
                phrase_template += ', '
                # break
            elif i[1] == 'JJ' or i[1] == 'NN' or i[1] == 'NNS':
                candidates.append(i[0])
        phrase_template = phrase_template[:-3]+'"'

        if ', ' in phrase_template:
            template_01 = phrase_template + ' are important phrases. '
        elif ' ' in phrase_template:
            template_01 = phrase_template + ' is an important phrase. '
        elif len(phrase_template)>3:
            template_01 = phrase_template + ' is an important word. '
        else:
            template_01 = 'There is no important phrase in this query. '
            # template_01 = ' '

        if len(candidates)>0:
            for candi in range(len(candidates)):
                word_template+=candidates[candi]+'", '
                # if candi >= 2:
                #     break
            word_template = word_template[:-2]

            if template_01 == 'There is no important phrase in this query. ':
                template_02 = 'But also pay attention to these words: ' + word_template
            else:
                template_02 = 'And also pay attention to these words: ' + word_template

            template = template_01+template_02
        else:
            template = template_01

        qa['template'] = template

    return qa_dataset

In [91]:
Phrase_squad = '../../data/SQuAD_V2/Phrase_SQUAD/'
for file in filelist:
    f = json.load(open(os.path.join(original_squad, file)))
    withPrompt = generate_phrase_template_for_domain(f)
    json.dump(withPrompt, open(os.path.join(Phrase_squad, file), 'w', encoding='utf-8'))

ValueError: Illegal chunk pattern: {++}

In [1]:
0.99*0.0001/(0.99*0.0001+0.01*0.9999)

0.00980392156862745

In [14]:
0.8067-0.6527+0.4847

0.6387